In [0]:
sc

<SparkContext master=spark://10.139.64.5:7077 appName=Databricks Shell>

In [0]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [0]:
spark=SparkSession.builder.appName("structured streaming").getOrCreate()

In [0]:
greenTaxiDF = spark.read.csv("/Volumes/databricksworkspace/default/greentaxi/2021_Green_Taxi_Trip_Data.csv",inferSchema=True,header=True)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2892038742555067>, line 1
----> 1 greenTaxiDF = spark.read.csv("/Volumes/databricksworkspace/default/greentaxi/2021_Green_Taxi_Trip_Data.csv",inferSchema=True,header=True)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:835, in DataFrameReader.csv(self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, positiveInf, negativeInf, dateFormat, timestampForma

In [0]:
greenTaxiDF = spark.read.csv("/Volumes/azuredatabricks_1405569260508774/default/greentaxi/2021_Green_Taxi_Trip_Data.csv", inferSchema=True, header=True)

# greenTaxiDF.repartition(10).write.mode("append").option("header","true").csv("/Volumes/azuredatabricks_1405569260508774/default/greentaxi/output/")

# greenTaxiDF.repartition(10).write.mode("append").option("header","true").parquet("/Volumes/azuredatabricks_1405569260508774/default/greentaxi/output/")

In [0]:
greenTaxiDF.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', StringType(), True), StructField('lpep_dropoff_datetime', StringType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', IntegerType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', StringType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('trip_type', IntegerType(), True), StructField('congestion_surcharge', Doubl

##### 1. Structured Streaming Transformation - Filter()

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

schema = StructType([
    StructField('VendorID', IntegerType(), True),
    StructField('lpep_pickup_datetime', StringType(), True),
    StructField('lpep_dropoff_datetime', StringType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('RatecodeID', IntegerType(), True),
    StructField('PULocationID', IntegerType(), True),
    StructField('DOLocationID', IntegerType(), True),
    StructField('passenger_count', IntegerType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('ehail_fee', StringType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('payment_type', IntegerType(), True),
    StructField('trip_type', IntegerType(), True),
    StructField('congestion_surcharge', DoubleType(), True)
])

# Define file paths
checkpoint_path = "/Volumes/azuredatabricks_1405569260508774/default/checkpoint/"
file_path = "/Volumes/azuredatabricks_1405569260508774/default/readstream/"
target_path = "/Volumes/azuredatabricks_1405569260508774/default/writestream/"

# Step 1: Create a ReadStream with explicit schema
taxiReadStream = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("cloudFiles.schemaLocation", checkpoint_path) \
    .schema(schema) \
    .load(file_path)

# Step 2: Add current timestamp as new column in taxiReadStream
taxiReadStream = taxiReadStream.withColumn("current_time", current_timestamp())

# Filter: Keep only trips with more than 1 passenger
filtered_df = taxiReadStream.filter(col('passenger_count') > 1)

# Select specific columns
selected_df = filtered_df.select("PULocationID", "DOLocationID", "trip_distance", "passenger_count", "fare_amount", "current_time")

# Add a new column: Calculate fare per mile
with_fare_per_mile_df = selected_df.withColumn("fare_per_mile", col("fare_amount") / col("trip_distance"))

# Step 4: Write the transformed data to Delta files
query = with_fare_per_mile_df.writeStream \
    .outputMode("append") \
    .format("csv") \
    .option("path", target_path) \
    .option("checkpointLocation", checkpoint_path) \
    .start()

# Await termination to keep the streaming query active
query.awaitTermination()


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
spark.read.csv('/Volumes/azuredatabricks_1405569260508774/default/writestream/').show(truncate=False)

##### 2. Structured Streaming Transformation - Filter(), WithColumn(), Aggregate(), groupBy()

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Define file paths
checkpoint_path = "/Volumes/azuredatabricks_1405569260508774/default/checkpoint/"
file_path = "/Volumes/azuredatabricks_1405569260508774/default/readstream/"
target_path = "/Volumes/azuredatabricks_1405569260508774/default/writestream/"

schema = StructType([
    StructField('VendorID', IntegerType(), True),
    StructField('lpep_pickup_datetime', StringType(), True),
    StructField('lpep_dropoff_datetime', StringType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('RatecodeID', IntegerType(), True),
    StructField('PULocationID', IntegerType(), True),
    StructField('DOLocationID', IntegerType(), True),
    StructField('passenger_count', IntegerType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('ehail_fee', StringType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('payment_type', IntegerType(), True),
    StructField('trip_type', IntegerType(), True),
    StructField('congestion_surcharge', DoubleType(), True)
])

# Step 1: Create a ReadStream with explicit schema
taxiReadStream = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("cloudFiles.schemaLocation", checkpoint_path) \
    .schema(schema) \
    .load(file_path)

# Step 2: Add current timestamp as new column in taxiReadStream
taxiReadStream = taxiReadStream.withColumn("current_time", current_timestamp())

# Filter: Keep only trips with more than 1 passenger
filtered_df = taxiReadStream.filter(col('passenger_count') > 0)

# Select specific columns
selected_df = filtered_df.select("PULocationID", "DOLocationID", "trip_distance", "passenger_count", "fare_amount", "current_time")

# Add a new column: Calculate fare per mile
with_fare_per_mile_df = selected_df.withColumn("fare_per_mile", col("fare_amount") / col("trip_distance"))

# GroupBy and Aggregate: Calculate average fare and total distance by PULocationID and DOLocationID within a 10-minute window

# Step 3: Perform the aggregation
# Note: The window is defined using the current_time column which sets the watermark on timestamp column.
# withWatermark allows to query, aggregate late-arriving data. In this example we set the watermark to 10 minutes. Any data older than 10 minutes relative to current timestamp will be ignored.
aggregated_df = with_fare_per_mile_df \
    .withWatermark("current_time", "1 hour") \
    .groupBy(col("PULocationID"), col("DOLocationID"), window(col("current_time"), "1 hour")) \
    .agg(
        avg("fare_amount").alias("average_fare"),
        sum("trip_distance").alias("total_distance"),
        count("*").alias("trip_count")
    ).orderBy(col("trip_count").desc())

# Step 4: Write the transformed data to Delta files
query = aggregated_df.writeStream \
    .outputMode("complete") \
    .format("delta") \
    .option("path", target_path) \
    .option("checkpointLocation", checkpoint_path) \
    .start()

# Await termination to keep the streaming query active
query.awaitTermination()

java.lang.Exception: Could not stop computations
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$4(DriverClient.scala:764)
	at scala.concurrent.Future.$anonfun$flatMap$1(Future.scala:307)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:54)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:77)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$2(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:271)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:267)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:46)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:43)
	at com.databricks.threading.ContextBoundRunnable.withAttributionContext(ContextBoundRunnable.scala:6)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$1(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.context.integrity.IntegrityCheckContext$ThreadLocalStorage$.withValue(IntegrityCheckContext.scala:73)
	at com.databricks.threading.ContextBoundRunnable.run(ContextBoundRunnable.scala:14)
	at com.databricks.threading.DatabricksExecutionContext$InstrumentedRunnable.run(DatabricksExecutionContext.scala:36)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:477)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)

In [0]:
spark.read.format("delta").load("/Volumes/azuredatabricks_1405569260508774/default/writestream/").show()

##### 3. Structured Streaming using Query Language

In [0]:
from pyspark.sql.functions import col, current_timestamp, window, avg, count, sum
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

# Define the schema explicitly
schema = StructType([
    StructField('VendorID', IntegerType(), True),
    StructField('lpep_pickup_datetime', StringType(), True),
    StructField('lpep_dropoff_datetime', StringType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('RatecodeID', IntegerType(), True),
    StructField('PULocationID', IntegerType(), True),
    StructField('DOLocationID', IntegerType(), True),
    StructField('passenger_count', IntegerType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('ehail_fee', StringType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('payment_type', IntegerType(), True),
    StructField('trip_type', IntegerType(), True),
    StructField('congestion_surcharge', DoubleType(), True)
])

# Define file paths
checkpoint_path = "/Volumes/azuredatabricks_1405569260508774/default/checkpoint/"
file_path = "/Volumes/azuredatabricks_1405569260508774/default/readstream/"
target_path = "/Volumes/azuredatabricks_1405569260508774/default/writestream/"

# Step 1: Create a ReadStream with explicit schema
taxiReadStream = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("cloudFiles.schemaLocation", checkpoint_path) \
    .schema(schema) \
    .load(file_path)

# Step 2: Add current timestamp
taxiReadStream = taxiReadStream.withColumn("current_time", current_timestamp())

# Step 3: Apply various transformations

# Filter: Keep only trips with more than 1 passenger
filtered_df = taxiReadStream.filter(col('passenger_count') > 0)

# Select specific columns
selected_df = filtered_df.select("PULocationID", "DOLocationID", "trip_distance", "passenger_count", "fare_amount", "current_time")

# Add a new column: Calculate fare per mile
with_fare_per_mile_df = selected_df.withColumn("fare_per_mile", col("fare_amount") / col("trip_distance"))

# Create a temporary view for the streaming DataFrame
with_fare_per_mile_df.createOrReplaceTempView("fare_per_mile_view")

# Define a query to run on the temporary view
sql_query = """
SELECT PULocationID, DOLocationID, AVG(fare_per_mile) as avg_fare_per_mile
FROM fare_per_mile_view
GROUP BY PULocationID, DOLocationID
ORDER BY avg_fare_per_mile DESC
LIMIT 10
"""

# Execute the query and get the result as a DataFrame
sql_query_df = spark.sql(sql_query)

# Start the streaming query to continuously run the SQL query
query = sql_query_df.writeStream \
    .outputMode("complete") \
    .format("delta") \
    .option("path", target_path) \
    .option("checkpointLocation", checkpoint_path) \
    .start()

# Await termination to keep the streaming query active
query.awaitTermination()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
spark.read.format("delta").load("/Volumes/azuredatabricks_1405569260508774/default/writestream/").show()